# Data Collection and Cleaning

For my project I am going to be analyzing traffic and specifically vehicle accidents that occured on a specified stretch of the San Bernardino Freeway. For the following data collection, 57 files for each of the 6 traffic features were manually exported from the Caltrans Performance Measurement System (PeMS) and 1 text file was exported containing all the weather data by hour from 2006 to the current date. The 2018 data was extracted using excel and then converted into a csv file.

In [1]:
%matplotlib
import pandas as pd
import numpy as np
pd.options.display.max_rows = 10

Using matplotlib backend: Qt5Agg


# Traffic Data

PeMS contains data for each day in 5 minute intervals; however, since I am going to be using 6 different features, manually extracting each data for each feature would not be feasible. Therefore, the data that I extracted was the time of day data. These reports contain the hourly measurements averaged over a selected week. Since I am going to be using all of 2018 as training data and the first month of 2019 as testing data, 57 weeks (csv files) were extracted independently for each of the 6 features. The files were extracted in order, therefore, the first file represents the first week of the year and so on.

**Visualize Traffic Data**

Each of the files contain the different postmiles from which the data was collected. Therefore, each time stamp contains multiple different observations. The times range from '0:00' to '24:00' representing one day; however, the metrics are averaged over an entire week. For example, the first row in the DataFrame below contains the average weekly speed during 0:00.

In [2]:
pd.read_csv('Traffic_Data_CSV/Speed/pems_output-1.csv')

,Time,Postmile (Abs),Postmile (CA),VDS,Speed,# Lane Points,% Observed
0,0:00,53.02,6.257,801269,69.17,288.0,66.7
1,0:00,53.75,6.980,806774,67.55,288.0,33.3
2,0:00,54.06,7.292,806751,67.82,360.0,66.7
3,0:00,54.55,7.788,826150,71.10,360.0,66.7
4,0:00,54.87,8.108,801286,65.68,288.0,50.0
...,...,...,...,...,...,...,...
259,23:00,55.31,8.543,801289,66.65,360.0,0.0
260,23:00,55.89,9.123,817213,68.98,360.0,66.7
261,23:00,56.97,10.200,818158,68.67,288.0,66.7
262,23:00,57.80,11.030,813079,69.30,288.0,66.7


**Extract Data**

The files collected from the PeMS dataset all have a similar format. They all contain the time, postmile, and the individual metric that is being evaluated. In addition, as noted before, each of the files were extracted in order. Therefore, by reading them in this order they will naturally represent the consecutive weeks of the year. Since each of the files can be processed in a similar manner, the function below is used to automate the entire reading process.

In [3]:
base_file = '/pems_output-'
base_directory = 'Traffic_Data_CSV/'
extension = '.csv'

In [4]:
def Extract_Data(directory, num_files, features_to_keep=False, add_week=True):
    
    dataframes = []
    path = base_directory + directory + base_file
    
    # loop through files
    for idx in range(1, num_files+1):
        df = pd.read_csv(path + str(idx) + extension)
        
        # only keep specified features
        if features_to_keep:
            df = df[features_to_keep]
            
        # add week as a feature if needed
        if add_week:
            df['Week'] = idx
            
        # add dataframe to list
        dataframes.append(df)
    
    # concatenate dataframes
    final_df = pd.concat(dataframes, ignore_index=True)
        
    return final_df

**Merge Data**

The entire extraction, cleaning and merging process is the same for all the files. <br>
Therefore, the function below is used to automate this entrire process of both reading <br>each file and merging them into one collective DataFrame.

In [5]:
# common features
base_features = ['Time', 'Postmile (Abs)']

# individual features
features = ['Speed', 'Flow', 'Occupancy', 'Truck Flow']
features_to_keep = []

# create list of the features needed for each file
for feature in features:
    keep = base_features + [feature]
    features_to_keep.append(keep)

In [6]:
features_to_keep

[['Time', 'Postmile (Abs)', 'Speed'],
 ['Time', 'Postmile (Abs)', 'Flow'],
 ['Time', 'Postmile (Abs)', 'Occupancy'],
 ['Time', 'Postmile (Abs)', 'Truck Flow']]

In [7]:
directories = ['Speed', 'Flow', 'Occupancy', 'Truck']

In [8]:
def Merge_Data(directories, num_files, feature_sets):
    
    final_df = Extract_Data(directories[0], num_files, feature_sets[0])
    common_features = base_features + ['Week']
    
    # create the different datasets
    for idx in range(1, len(feature_sets)):
        
        # read the files to create dataframe
        new_df = Extract_Data(directories[idx], num_files, feature_sets[idx])
        
        # merge datasets
        final_df = final_df.merge(new_df, 
                                  left_on=common_features, 
                                  right_on=common_features)
        
    return final_df

**Create Dataset**

In [9]:
traffic_df = Merge_Data(directories, 57, features_to_keep)

In [10]:
traffic_df

,Time,Postmile (Abs),Speed,Week,Flow,Occupancy,Truck Flow
0,0:00,53.02,69.17,1,1634,0.03,155
1,0:00,53.75,67.55,1,1646,0.03,63
2,0:00,54.06,67.82,1,1584,0.05,3
3,0:00,54.55,71.10,1,1671,0.02,235
4,0:00,54.87,65.68,1,1590,0.04,272
...,...,...,...,...,...,...,...
15043,23:00,55.31,65.42,57,2541,0.05,81
15044,23:00,55.89,66.72,57,3294,0.10,943
15045,23:00,56.97,68.26,57,1405,0.02,27
15046,23:00,57.80,68.30,57,1872,0.04,280


**Collect & Clean Miles Traveled Data**

The miles traveled files are slightly different in that they do not contain <br>
the postmiles. The miles traveled are simply averaged throughout the entire <br>
freeway stretch. Therefore, these files are extracted and cleaned independently.

In [11]:
pd.read_csv('Traffic_Data_CSV/Traveled/pems_output-1.csv')

,Time,Minimum,Mean,Maximum,# Lane Points,% Observed
0,00:00,"4,850,355.45","5,817,388.00","7,240,637.80",2536803,60.4
1,01:00,"3,526,999.80","4,739,787.51","7,823,452.51",2536812,61.7
2,02:00,"3,202,037.51","4,047,091.63","5,847,196.53",2536246,62.4
3,03:00,"3,923,831.25","4,302,043.01","4,667,676.07",2536608,62.3
4,04:00,"4,761,272.61","7,039,788.75","8,172,126.18",2537016,62.2
...,...,...,...,...,...,...
19,19:00,"18,213,128.54","19,194,415.83","21,099,115.44",2537220,61.8
20,20:00,"15,139,915.53","16,040,107.05","17,788,464.73",2536812,61.7
21,21:00,"13,118,238.76","14,248,050.98","15,760,962.91",2537016,60.0
22,22:00,"10,385,222.41","11,719,256.04","14,195,282.70",2536764,59.9


In [12]:
# Mean is the average miles traveled
features_to_keep = ['Time', 'Mean']
traveled_df = Extract_Data('Traveled', 57, features_to_keep)

In [13]:
# remove the leading zero from the Time column
traveled_df.Time = traveled_df.Time.str.lstrip('0')
traveled_df.Time = traveled_df.Time.replace(':00', '0:00')

# convert the mean to a numeric values
traveled_df.Mean = pd.to_numeric(
    traveled_df.Mean.str.replace('[\,.*?]', ''))

# rename the columns to match the first DataFrame
traveled_df.columns=['Time', 'Miles Traveled (Mean)', 'Week']

In [14]:
traveled_df

,Time,Miles Traveled (Mean),Week
0,0:00,581738800,1
1,1:00,473978751,1
2,2:00,404709163,1
3,3:00,430204301,1
4,4:00,703978875,1
...,...,...,...
1363,19:00,1812596562,57
1364,20:00,1504245084,57
1365,21:00,1277498028,57
1366,22:00,959771523,57


**Merge Datasets**

In [15]:
traffic_df = traffic_df.merge(traveled_df, left_on=['Week', 'Time'], right_on=['Week', 'Time'])

In [16]:
traffic_df

,Time,Postmile (Abs),Speed,Week,Flow,Occupancy,Truck Flow,Miles Traveled (Mean)
0,0:00,53.02,69.17,1,1634,0.03,155,581738800
1,0:00,53.75,67.55,1,1646,0.03,63,581738800
2,0:00,54.06,67.82,1,1584,0.05,3,581738800
3,0:00,54.55,71.10,1,1671,0.02,235,581738800
4,0:00,54.87,65.68,1,1590,0.04,272,581738800
...,...,...,...,...,...,...,...,...
15043,23:00,55.31,65.42,57,2541,0.05,81,658290165
15044,23:00,55.89,66.72,57,3294,0.10,943,658290165
15045,23:00,56.97,68.26,57,1405,0.02,27,658290165
15046,23:00,57.80,68.30,57,1872,0.04,280,658290165


**Collect & Clean Traffic Accidents Data**

The accidents files are also different in that they are individual accounts <br>
by day. They do however include the postmile at which these accidents occur (Abs PM).

In [17]:
pd.read_csv('Traffic_Data_CSV/accidents/pems_output-1.csv')

,Incident Id,Start Time,Duration (mins),Freeway,CA PM,Abs PM,Source,AREA,LOCATION,DESCRIPTION
0,"17,691,256.00",01-01-18 18:02,14.0,I10-E,7.735,54.5,CHP,Rancho Cucamonga,I10 E Wo - Haven Ave,20002-Hit and Run No Injuries
1,"17,692,459.00",01-02-18 14:28,30.0,I10-E,7.735,54.5,CHP,Rancho Cucamonga,I10 E EO - ARCHIBALD AVE,1182-Trfc Collision-No Inj
2,"17,692,585.00",01-02-18 15:33,236.0,I10-E,7.135,53.9,CHP,Rancho Cucamonga,I10 E - I10 E Archibald Aveo Fr,1179-Trfc Collision-1141 Enrt
3,"17,692,627.00",01-02-18 15:53,20.0,I10-E,6.535,53.3,CHP,San Bernardino FSP,I10 E Wo - Archibald Ave,1182-Trfc Collision-No Inj
4,"17,692,628.00",01-02-18 15:52,1.0,I10-E,6.535,53.3,CHP,Rancho Cucamonga,I10 E Wo - Archibald Ave,1182-Trfc Collision-No Inj
...,...,...,...,...,...,...,...,...,...,...
9,"17,696,512.00",01-04-18 15:10,24.0,I10-E,8.635,55.4,CHP,Rancho Cucamonga,I10 E Wo - Milliken Ave,1183-Trfc Collision-Unkn Inj
10,"17,696,637.00",01-04-18 16:17,32.0,I10-E,10.435,57.2,CHP,San Bernardino FSP,I10 E Eo - I15,1183-Trfc Collision-Unkn Inj
11,"17,696,646.00",01-04-18 16:23,27.0,I10-E,11.035,57.8,CHP,San Bernardino FSP,I10 E - Etiwanda Ave,1183-Trfc Collision-Unkn Inj
12,"17,700,943.00",01-06-18 17:43,31.0,I10-E,8.635,55.4,CHP,San Bernardino FSP,I10 E Wo - Milliken Ave,20002-Hit and Run No Injuries


In [18]:
accidents_df = Extract_Data('accidents', 57, ['Abs PM', 'Start Time'])

In [19]:
# Clean the Start Time column and create a new Time column for compatability
accidents_df['Time'] = (accidents_df['Start Time']
                        .str.split(' ', expand=True)[1]
                        .str.split(':', expand=True)[0] + ':00')

# drop old time column
accidents_df.drop('Start Time', axis=1, inplace=True)

In [20]:
accidents_df.Time.unique()

array(['18:00', '14:00', '15:00', '17:00', '02:00', '23:00', '16:00',
       '08:00', '19:00', '13:00', '01:00', '07:00', '20:00', '06:00',
       '12:00', '11:00', '05:00', '21:00', '10:00', '22:00', '04:00',
       '03:00', '09:00', '00:00'], dtype=object)

In [21]:
traffic_df.Time.unique()

array(['0:00', '1:00', '2:00', '3:00', '4:00', '5:00', '6:00', '7:00',
       '8:00', '9:00', '10:00', '11:00', '12:00', '13:00', '14:00',
       '15:00', '16:00', '17:00', '18:00', '19:00', '20:00', '21:00',
       '22:00', '23:00'], dtype=object)

In [22]:
# These are the columns that match with the DataFrame containing
# other features (traffic_df)
accidents_df.columns=['Postmile (Abs)', 'Week', 'Time']

In [23]:
# Remove the leading zero for compatability purposes
accidents_df.Time = (accidents_df.Time
                     .str.lstrip('0')
                     .str.replace('^:00$', '0:00', regex=True))
accidents_df

,Postmile (Abs),Week,Time
0,54.5,1,18:00
1,54.5,1,14:00
2,53.9,1,15:00
3,53.3,1,15:00
4,53.3,1,15:00
...,...,...,...
679,54.7,56,3:00
680,54.5,56,13:00
681,57.4,57,11:00
682,57.8,57,18:00


**Mege Datasets**

The postmiles between the traffic data and the accident data are slightly off. <br>
In order to merge the two datasets, the postmiles from the accidents are going to <br>
be translated to the postmiles from the traffic data. In order to translate the postmiles, <br>
the accidents postmiles are mapped to their closest postmile in the traffic dataset. 

In [24]:
traffic_df['Postmile (Abs)'].unique()

array([53.02, 53.75, 54.06, 54.55, 54.87, 54.99, 55.31, 55.89, 56.97,
       57.8 , 57.86])

In [25]:
accidents_df['Postmile (Abs)'].unique()

array([54.5, 53.9, 53.3, 57.8, 56. , 57.4, 55.4, 57.2, 53.2, 54.9, 54.2,
       53.5, 56.2, 57.9, 56.7, 56.6, 56.8, 57.5, 55.6, 55.2, 54.7, 55. ,
       55.7, 53.7])

In [26]:
nearest = {}
postmiles = traffic_df['Postmile (Abs)'].unique()

# map postmiles from accidents to postmiles from traffic data
for value in accidents_df['Postmile (Abs)'].unique():
    
    # postmile with smallest difference is the nearest
    index = (np.abs(postmiles-value)).argmin()
    
    # append mapping
    nearest[value] = postmiles[index]
    

In [27]:
nearest

{54.5: 54.55,
 53.9: 53.75,
 53.3: 53.02,
 57.8: 57.8,
 56.0: 55.89,
 57.4: 57.8,
 55.4: 55.31,
 57.2: 56.97,
 53.2: 53.02,
 54.9: 54.87,
 54.2: 54.06,
 53.5: 53.75,
 56.2: 55.89,
 57.9: 57.86,
 56.7: 56.97,
 56.6: 56.97,
 56.8: 56.97,
 57.5: 57.8,
 55.6: 55.31,
 55.2: 55.31,
 54.7: 54.55,
 55.0: 54.99,
 55.7: 55.89,
 53.7: 53.75}

In [28]:
# replace accidents postmiles with their corresponding traffic postmiles
accidents_df['Postmile (Abs)'] = accidents_df['Postmile (Abs)'].replace(nearest)

In [29]:
accidents_df

,Postmile (Abs),Week,Time
0,54.55,1,18:00
1,54.55,1,14:00
2,53.75,1,15:00
3,53.02,1,15:00
4,53.02,1,15:00
...,...,...,...
679,54.55,56,3:00
680,54.55,56,13:00
681,57.80,57,11:00
682,57.80,57,18:00


The accidents DataFrame cannot be merged with the traffic DataFrame since <br>
multiple accidents occured at the same postmiles. In order to merge, the accidents <br>
have to be accumulated.

In [30]:
accident_counts = pd.DataFrame(accidents_df.groupby(['Week', 'Time'])['Postmile (Abs)']
                               .value_counts())
accident_counts.columns=['Accidents']
accident_counts

Accidents
Week Time  Postmile (Abs)           
1    14:00 54.55                   1
     15:00 53.02                   2
           53.75                   1
           55.31                   1
     16:00 56.97                   1
...                              ...
56   3:00  54.55                   1
     6:00  53.02                   1
           55.31                   1
57   11:00 57.80                   1
     18:00 57.80                   2

[517 rows x 1 columns]

In [31]:
traffic_df = traffic_df.merge(accident_counts, 
                              how='left',
                              left_on=['Week', 'Time', 'Postmile (Abs)'], 
                              right_index=True).fillna(0)

In [32]:
traffic_df

,Time,Postmile (Abs),Speed,Week,Flow,Occupancy,Truck Flow,Miles Traveled (Mean),Accidents
0,0:00,53.02,69.17,1,1634,0.03,155,581738800,0.0
1,0:00,53.75,67.55,1,1646,0.03,63,581738800,0.0
2,0:00,54.06,67.82,1,1584,0.05,3,581738800,0.0
3,0:00,54.55,71.10,1,1671,0.02,235,581738800,0.0
4,0:00,54.87,65.68,1,1590,0.04,272,581738800,0.0
...,...,...,...,...,...,...,...,...,...
15043,23:00,55.31,65.42,57,2541,0.05,81,658290165,0.0
15044,23:00,55.89,66.72,57,3294,0.10,943,658290165,0.0
15045,23:00,56.97,68.26,57,1405,0.02,27,658290165,0.0
15046,23:00,57.80,68.30,57,1872,0.04,280,658290165,0.0


# Weather Data

**Collect & Clean Weather Data**

As mentioned in the beginning, the weather data was collected independently. <br>
The data includes both the hour and date, which are going to be used to merge.

In [33]:
weather_df = pd.read_csv('Traffic_Data_CSV/Weather/weather.csv')

# remove missing values
weather_df.dropna(inplace=True)
weather_df.reset_index(drop=True, inplace=True)
weather_df

,Site,Date,Hour,Temperature,Dewpoint,RH,WindDir,Windspeed,CldFrac,MSLP,Weather,Precip,Source
0,KRIV,1/1/18 0:00,0,40,37,89,290,3,0.0,30.15,CLR,0.0,NWS
1,KRIV,1/1/18 0:00,1,35,31,85,0,0,0.0,30.14,CLR,0.0,NWS
2,KRIV,1/1/18 0:00,2,38,33,82,0,0,0.0,30.13,CLR,0.0,NWS
3,KRIV,1/1/18 0:00,3,35,32,89,0,0,0.0,30.13,CLR,0.0,NWS
4,KRIV,1/1/18 0:00,4,37,32,82,0,0,0.0,30.13,CLR,0.0,NWS
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9408,KRIV,1/31/19 0:00,19,51,51,100,40,8,0.4,29.94,SCT,0.0,NWS
9409,KRIV,1/31/19 0:00,20,50,50,100,210,3,0.0,29.93,CLR,0.0,NWS
9410,KRIV,1/31/19 0:00,21,47,47,100,0,0,0.0,29.95,CLR,0.0,NWS
9411,KRIV,1/31/19 0:00,22,45,45,100,0,0,0.0,29.95,CLR,0.0,NWS


In [34]:
# extract only the meaningful weather metrics
weather_df = weather_df[['Date', 'Hour', 'Temperature', 'Windspeed', 'Weather', 'Precip']]

In [35]:
weather_df

,Date,Hour,Temperature,Windspeed,Weather,Precip
0,1/1/18 0:00,0,40,3,CLR,0.0
1,1/1/18 0:00,1,35,0,CLR,0.0
2,1/1/18 0:00,2,38,0,CLR,0.0
3,1/1/18 0:00,3,35,0,CLR,0.0
4,1/1/18 0:00,4,37,0,CLR,0.0
...,...,...,...,...,...,...
9408,1/31/19 0:00,19,51,8,SCT,0.0
9409,1/31/19 0:00,20,50,3,CLR,0.0
9410,1/31/19 0:00,21,47,0,CLR,0.0
9411,1/31/19 0:00,22,45,0,CLR,0.0


The labels below are weather acronyms (for example CLR stands for clear). <br>
Since they are categorical variables, they are converted to dummy variables.

In [36]:
conditions = pd.get_dummies(weather_df.Weather)
conditions.columns

Index(['#NAME?', '9999', '9999 -RA', 'BKN', 'BR', 'BR VCSH', 'CLR', 'DU',
       'FEW', 'FG', 'HZ', 'OVC', 'RA', 'RA BR', 'SCT', 'TS', 'TSRA BR', 'VCSH',
       'VCTS', 'VCTS +RA BR'],
      dtype='object')

In [37]:
# disregard the first three columns (unknown values)
weather_df = weather_df.merge(conditions.iloc[:,3:],left_index=True, right_index=True)

In [38]:
# 'Weather' was converted to dummies, can disregard it
weather_df.drop(['Weather'], axis=1,inplace=True)

In [39]:
weather_df

,Date,Hour,Temperature,Windspeed,Precip,BKN,BR,BR VCSH,CLR,DU,...,HZ,OVC,RA,RA BR,SCT,TS,TSRA BR,VCSH,VCTS,VCTS +RA BR
0,1/1/18 0:00,0,40,3,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1/1/18 0:00,1,35,0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1/1/18 0:00,2,38,0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1/1/18 0:00,3,35,0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1/1/18 0:00,4,37,0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9408,1/31/19 0:00,19,51,8,0.0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
9409,1/31/19 0:00,20,50,3,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9410,1/31/19 0:00,21,47,0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9411,1/31/19 0:00,22,45,0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


The weather DataFrame is going to be merged onto the traffic DataFrame using the <br>
'Week' and 'Time' . Therefore, the 'Date' column from the weather DataFrame is going <br>
to be cleaned to compute the week. The 'Hour' is then going to be cleaned to <br>
match the 'Time' column of the traffic DataFrame.

In [40]:
weather_df.Date = weather_df.Date.str.split(' ', expand=True)[0]
weather_df

,Date,Hour,Temperature,Windspeed,Precip,BKN,BR,BR VCSH,CLR,DU,...,HZ,OVC,RA,RA BR,SCT,TS,TSRA BR,VCSH,VCTS,VCTS +RA BR
0,1/1/18,0,40,3,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1/1/18,1,35,0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1/1/18,2,38,0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1/1/18,3,35,0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,1/1/18,4,37,0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9408,1/31/19,19,51,8,0.0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
9409,1/31/19,20,50,3,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9410,1/31/19,21,47,0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9411,1/31/19,22,45,0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
weather_df.dtypes

Date            object
Hour             int64
Temperature      int64
Windspeed        int64
Precip         float64
                ...   
TS               uint8
TSRA BR          uint8
VCSH             uint8
VCTS             uint8
VCTS +RA BR      uint8
Length: 22, dtype: object

In order to compute the weeks, the days are iterated and a count is kept to keep <br>
track of the number of days that have passed. After the 7th day, the week is changed. <br>
The week of every day is appened into one list. Since the weather is computed hourly, <br>
'current_day' is used to keep track of when a new day begins.

In [42]:
# manually initialize the first week since
# it does not contain exactly 7 days
weeks = [1]*6*24
week=2
current_day = None
day_count = 0

for day in weather_df.Date[6*24:]:
    
    # check if its a new day
    if day != current_day:
        current_day = day
        day_count += 1
        
    # check if its a new week
    if day_count > 7:
        week += 1
        day_count = 1
        
    # add current Week
    weeks.append(week)

In [43]:
len(weeks), np.unique(weeks)

(9413,
 array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
        35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
        52, 53, 54, 55, 56, 57]))

In [44]:
# create new 'Week' column
weather_df['Week'] = weeks
weather_df

,Date,Hour,Temperature,Windspeed,Precip,BKN,BR,BR VCSH,CLR,DU,...,OVC,RA,RA BR,SCT,TS,TSRA BR,VCSH,VCTS,VCTS +RA BR,Week
0,1/1/18,0,40,3,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,1/1/18,1,35,0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,1/1/18,2,38,0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
3,1/1/18,3,35,0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,1/1/18,4,37,0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9408,1/31/19,19,51,8,0.0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,57
9409,1/31/19,20,50,3,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,57
9410,1/31/19,21,47,0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,57
9411,1/31/19,22,45,0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,57


In [45]:
# append suffix to match 'Time' in the traffic DataFrame
weather_df['Time'] = weather_df['Hour'].astype(str) + ':00'

In [46]:
weather_df

,Date,Hour,Temperature,Windspeed,Precip,BKN,BR,BR VCSH,CLR,DU,...,RA,RA BR,SCT,TS,TSRA BR,VCSH,VCTS,VCTS +RA BR,Week,Time
0,1/1/18,0,40,3,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0:00
1,1/1/18,1,35,0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1:00
2,1/1/18,2,38,0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,2:00
3,1/1/18,3,35,0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,3:00
4,1/1/18,4,37,0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,4:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9408,1/31/19,19,51,8,0.0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,57,19:00
9409,1/31/19,20,50,3,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,57,20:00
9410,1/31/19,21,47,0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,57,21:00
9411,1/31/19,22,45,0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,57,22:00


In [47]:
# remove 'Date' and 'Hour'
features = weather_df.columns[2:-2]
features

Index(['Temperature', 'Windspeed', 'Precip', 'BKN', 'BR', 'BR VCSH', 'CLR',
       'DU', 'FEW', 'FG', 'HZ', 'OVC', 'RA', 'RA BR', 'SCT', 'TS', 'TSRA BR',
       'VCSH', 'VCTS', 'VCTS +RA BR'],
      dtype='object')

Since each observation in the traffic DataFrame is a weekly average, the hour <br>
observations from the weather DataFrame have to be average over the entire week.

In [48]:
# performs groupby for multiple features and appends 
# the resulting DataFrames to a list
def mult_groupby(df, grouping_feat, features, metric):
    dataframes = []
    for feature in features:
        if metric == 'sum':
            dataframe = pd.DataFrame(df.groupby(grouping_feat)[feature].sum())
        else:
            dataframe = pd.DataFrame(df.groupby(grouping_feat)[feature].mean())
        dataframes.append(dataframe)
    return dataframes

In [49]:
# Weekly Averages (metric features)
dfs_1 = mult_groupby(weather_df,
                     ['Week', 'Time'],
                     features[:3],
                     'mean')

# weekly counts (dummy variables)
dfs_2 = mult_groupby(weather_df,
                     ['Week', 'Time'],
                     features[3:],
                     'sum')

**Merge Datasets**

In [50]:
# Merge the weather metric features
for dataframe in dfs_1:
    traffic_df = traffic_df.merge(dataframe,
                                  how='left', 
                                  left_on=['Week', 'Time'],
                                  right_index=True)

In [51]:
# Merge the weather dummy variables
for dataframe in dfs_2:
    traffic_df = traffic_df.merge(dataframe,
                                  how='left', 
                                  left_on=['Week', 'Time'],
                                  right_index=True)

# Final Dataset

In [52]:
traffic_df

,Time,Postmile (Abs),Speed,Week,Flow,Occupancy,Truck Flow,Miles Traveled (Mean),Accidents,Temperature,...,HZ,OVC,RA,RA BR,SCT,TS,TSRA BR,VCSH,VCTS,VCTS +RA BR
0,0:00,53.02,69.17,1,1634,0.03,155,581738800,0.0,48.833333,...,0,0,0,0,1,0,0,0,0,0
1,0:00,53.75,67.55,1,1646,0.03,63,581738800,0.0,48.833333,...,0,0,0,0,1,0,0,0,0,0
2,0:00,54.06,67.82,1,1584,0.05,3,581738800,0.0,48.833333,...,0,0,0,0,1,0,0,0,0,0
3,0:00,54.55,71.10,1,1671,0.02,235,581738800,0.0,48.833333,...,0,0,0,0,1,0,0,0,0,0
4,0:00,54.87,65.68,1,1590,0.04,272,581738800,0.0,48.833333,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15043,23:00,55.31,65.42,57,2541,0.05,81,658290165,0.0,47.000000,...,0,0,0,0,1,0,0,0,0,0
15044,23:00,55.89,66.72,57,3294,0.10,943,658290165,0.0,47.000000,...,0,0,0,0,1,0,0,0,0,0
15045,23:00,56.97,68.26,57,1405,0.02,27,658290165,0.0,47.000000,...,0,0,0,0,1,0,0,0,0,0
15046,23:00,57.80,68.30,57,1872,0.04,280,658290165,0.0,47.000000,...,0,0,0,0,1,0,0,0,0,0


# Export

In [53]:
Train_df = traffic_df[traffic_df.Week < 53]
Test_df = traffic_df[traffic_df.Week>=53]

In [54]:
Train_df.to_csv('Traffic_Data_Train', sep=',', index=False)
Test_df.to_csv('Traffic_Data_Test', sep=',', index=False)